## PRAKTIKUM 6

Import & helper

In [ ]:
import os, glob, time, math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

import kagglehub
import faiss
from annoy import AnnoyIndex
import hnswlib

np.random.seed(42)

def recall_at_k(exact_idx, approx_idx):
    # exact_idx, approx_idx shape: (Q, k)
    inter = [len(set(e).intersection(set(a))) for e,a in zip(exact_idx, approx_idx)]
    return np.mean(np.array(inter) / exact_idx.shape[1])

def timeit(fn, *args, **kwargs):
    t0 = time.time()
    out = fn(*args, **kwargs)
    return out, time.time() - t0


Download dataset via kagglehub & load CSV

In [ ]:
path = kagglehub.dataset_download("bwandowando/spotify-songs-with-attributes-and-lyrics")
print("Downloaded to:", path)

# Cari CSV yang paling relevan (fallback ke yang pertama)
csvs = sorted(glob.glob(os.path.join(path, "**/*.csv"), recursive=True))
assert len(csvs) > 0, "CSV file not found in the Kaggle dataset path."

# Prioritaskan nama yang mengandung 'spotify' atau 'songs'
pri = [p for p in csvs if "spotify" in os.path.basename(p).lower() or "song" in os.path.basename(p).lower()]
csv_file = pri[0] if len(pri) else csvs[0]
print("Using CSV:", csv_file)

df = pd.read_csv(csv_file)
print(df.shape)
df.head(3)


100%|██████████| 894M/894M [00:30<00:00, 30.4MB/s]

Extracting files...


Downloaded to: /root/.cache/kagglehub/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/versions/19
Using CSV: /root/.cache/kagglehub/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/versions/19/songs_with_attributes_and_lyrics.csv
(955320, 17)


,id,name,album_name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,lyrics
0,0Prct5TDjAnEgIqbxcldY9,!,UNDEN!ABLE,['HELLYEAH'],0.415,0.6050,7,-11.157,1,0.0575,0.00116,0.838,0.471,0.193,100.059,79500.0,"He said he came from Jamaica,\n he owned a cou..."
1,2ASl4wirkeYm3OWZxXKYuq,!!,NaN,Yxngxr1,0.788,0.6480,7,-9.135,0,0.3150,0.90000,0.000,0.176,0.287,79.998,114000.0,"Fucked a bitch, now she running with my kids\n..."
2,69lcggVPmOr9cvPx9kLiiN,!!! - Interlude,Where I Belong EP,['Glowie'],0.000,0.0354,7,-20.151,0,0.0000,0.90800,0.000,0.479,0.000,0.000,11413.0,"Oh, my God, I'm going crazy\n"


Pilih fitur & preprocessing

In [ ]:
# Sesuaikan nama kolom jika berbeda—kode ini akan memfilter yang tersedia saja
candidate_features = [
    'danceability','energy','loudness','speechiness',
    'acousticness','instrumentalness','liveness','valence','tempo'
]
features = [c for c in candidate_features if c in df.columns]
assert len(features) >= 5, f"Tidak cukup fitur ditemukan. Ditemukan: {features}"

X = df[features].astype(np.float32).fillna(0).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X).astype(np.float32)

N, D = X_scaled.shape
print(f"N={N:,}, D={D}, features={features}")


N=955,320, D=9, features=['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']


Parameter eksperimen

In [ ]:
# K neighbor
k = 10

# Jumlah query untuk evaluasi (subset agar cepat). Naikkan kalau mau lebih akurat.
Q = min(2000, N)     # mis. 2000 query acak

# Annoy
N_TREES = 10

# HNSW
M = 16
EF_CONSTRUCTION = 200
EF_QUERY = 200

# FAISS IVF
NLIST = max(32, int(math.sqrt(N)))   # heuristik umum
NPROBE = min(64, max(8, int(NLIST*0.1)))  # 10% nlist


Ambil subset query & ground truth exact (sklearn brute)

In [ ]:
# Pilih Q baris acak untuk dijadikan query
rng = np.random.default_rng(123)
q_idx = rng.choice(N, size=Q, replace=False)
QX = X_scaled[q_idx]

# Exact (sklearn brute)
(nn_fit, t_build_exact) = timeit(
    lambda X: NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean').fit(X),
    X_scaled
)
((dist_exact, idx_exact), t_query_exact) = timeit(nn_fit.kneighbors, QX)

print(f"[Exact] build={t_build_exact:.3f}s, query={t_query_exact:.3f}s, total={t_build_exact+t_query_exact:.3f}s")


[Exact] build=0.024s, query=9.685s, total=9.709s


Annoy (euclidean)

In [ ]:
# Build
f = D
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v.tolist())

_, t_build_annoy = timeit(index_annoy.build, N_TREES)

# Query untuk subset Q
def annoy_batch_query(index, QX, k):
    out = []
    for v in QX:
        out.append(index.get_nns_by_vector(v.tolist(), k))
    return np.array(out, dtype=int)

(idx_annoy, t_query_annoy) = timeit(annoy_batch_query, index_annoy, QX, k)
rec_annoy = recall_at_k(idx_exact, idx_annoy)

print(f"[Annoy] trees={N_TREES} build={t_build_annoy:.3f}s, query={t_query_annoy:.3f}s, recall@{k}={rec_annoy:.3f}")


[Annoy] trees=10 build=14.430s, query=0.093s, recall@10=0.827


HNSW (L2)

In [ ]:
p = hnswlib.Index(space='l2', dim=D)
_, t_build_hnsw_init = timeit(p.init_index, max_elements=N, ef_construction=EF_CONSTRUCTION, M=M)
_, t_build_hnsw_add  = timeit(p.add_items, X_scaled, np.arange(N, dtype=np.int32))
p.set_ef(EF_QUERY)

# Batch query
((idx_hnsw, dist_hnsw), t_query_hnsw) = timeit(p.knn_query, QX, k=k)
rec_hnsw = recall_at_k(idx_exact, idx_hnsw)

print(f"[HNSW] M={M} efC={EF_CONSTRUCTION} efQ={EF_QUERY} "
      f"build={t_build_hnsw_init+t_build_hnsw_add:.3f}s, query={t_query_hnsw:.3f}s, recall@{k}={rec_hnsw:.3f}")


[HNSW] M=16 efC=200 efQ=200 build=173.610s, query=0.416s, recall@10=0.994


FAISS IVF-Flat (L2)

In [ ]:
d = D
quantizer = faiss.IndexFlatL2(d)
index_ivf = faiss.IndexIVFFlat(quantizer, d, NLIST, faiss.METRIC_L2)

# Train + add
_, t_train = timeit(index_ivf.train, X_scaled)
_, t_add   = timeit(index_ivf.add, X_scaled)
index_ivf.nprobe = NPROBE

# Query
((D_ivf, I_ivf), t_query_ivf) = timeit(index_ivf.search, QX, k)
rec_ivf = recall_at_k(idx_exact, I_ivf)

print(f"[FAISS IVF] nlist={NLIST} nprobe={NPROBE} train={t_train:.3f}s add={t_add:.3f}s "
      f"query={t_query_ivf:.3f}s recall@{k}={rec_ivf:.3f}")


[FAISS IVF] nlist=977 nprobe=64 train=1.313s add=0.499s query=0.935s recall@10=0.998


Ringkasan hasil

In [ ]:
summary = pd.DataFrame([
    {"Method":"Exact (sklearn brute)", "Build/Train (s)": round(t_build_exact,3), "Query (s)": round(t_query_exact,3), "Recall@k": 1.000},
    {"Method":f"Annoy (trees={N_TREES})", "Build/Train (s)": round(t_build_annoy,3), "Query (s)": round(t_query_annoy,3), "Recall@k": round(rec_annoy,3)},
    {"Method":f"HNSW (M={M}, efQ={EF_QUERY})", "Build/Train (s)": round(t_build_hnsw_init+t_build_hnsw_add,3), "Query (s)": round(t_query_hnsw,3), "Recall@k": round(rec_hnsw,3)},
    {"Method":f"FAISS IVF (nlist={NLIST}, nprobe={NPROBE})", "Build/Train (s)": round(t_train+t_add,3), "Query (s)": round(t_query_ivf,3), "Recall@k": round(rec_ivf,3)},
])
summary


,Method,Build/Train (s),Query (s),Recall@k
0,Exact (sklearn brute),0.024,9.685,1.000
1,Annoy (trees=10),14.430,0.093,0.827
2,"HNSW (M=16, efQ=200)",173.610,0.416,0.994
3,"FAISS IVF (nlist=977, nprobe=64)",1.813,0.935,0.998
